In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

print(os.getcwd())

/content


In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661677 sha256=d809cb86cb85c2ed1db9ca3c8a355ef00ca9485fff519fa015797ab8e95f1642
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00


In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-jfsg7554/kobert-tokenizer_1216cc4a0dbd4e62a9b250cee954e989
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-jfsg7554/kobert-tokenizer_1216cc4a0dbd4e62a9b250cee954e989
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=bf0c4aa68dc20c4f407e0f5828e2565297162a55aa1780b39e08669c89e554e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-n5fwsss4/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
# GPU 사용 시
device = torch.device("cuda:0")

In [8]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [9]:
import pandas as pd
train_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
dev_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/dev.hate.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.hate.no_label.csv')

In [10]:
LABEL_DIC = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

In [11]:
train_data['label'] = train_data['label'].map(lambda x: LABEL_DIC[x])
dev_data['label'] = dev_data['label'].map(lambda x: LABEL_DIC[x])

In [12]:
train_data_list = []
for q, label in zip(train_data['comments'], train_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train_data_list.append(data)

In [13]:
dev_data_list = []
for q, label in zip(dev_data['comments'], dev_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    dev_data_list.append(data)

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [42]:
# Setting parameters

max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [43]:
tok = tokenizer.tokenize

train_dataset = BERTDataset(train_data_list, 0, 1, tok, vocab, max_len, True, False)
dev_dataset = BERTDataset(dev_data_list, 0, 1, tok, vocab, max_len, True, False)

In [44]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=4)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [45]:
#kobert 학습모델 만들기

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3,   ##클래스 수 조정##
                 dr_rate = None,
                 params = None):

        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)

        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        pooler = self.bert(input_ids = token_ids, attention_mask = attention_mask.float().to(token_ids.device))[0][:,0]
        # distilbert는 pooler를 return하지 않음. 따라서 last_hidden_state로 부터 직접 pooler를 추출함

        if self.dr_rate:
            out = self.dropout(pooler)

        return self.classifier(out)

In [46]:
model = BERTClassifier(bertmodel, dr_rate = 0.1).to(device)
# defult = 0.5

In [47]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [48]:
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [49]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [50]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    print(token_ids)
    print(valid_length)
    print(segment_ids)
    print(label)
    break

<ipython-input-50-2a6bca869dc4>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/124 [00:00<?, ?it/s]

tensor([[   2,  522, 7906,  ...,    1,    1,    1],
        [   2,  517,   54,  ...,    1,    1,    1],
        [   2,  517,   55,  ...,    1,    1,    1],
        ...,
        [   2,  592, 5698,  ...,    1,    1,    1],
        [   2,  592, 5808,  ...,    1,    1,    1],
        [   2,  592, 5808,  ...,    1,    1,    1]], dtype=torch.int32)
tensor([37, 41, 49, 22, 36, 11, 50, 25, 23, 39, 22, 23, 15, 29, 13, 36, 27, 35,
        42, 46, 33, 37, 13, 14, 28, 30, 11, 25, 11, 28, 21, 47, 31, 46, 29, 64,
        64, 35,  9, 14, 22, 36, 26, 64, 11, 49, 12,  8, 10, 22, 62, 51, 39,  8,
        42, 23, 24, 43, 14, 29, 51, 26, 24, 16], dtype=torch.int32)
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)
tensor([2, 0, 2, 0, 2, 0, 2, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 2, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0

In [51]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(dev_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length)
        test_acc += calc_accuracy(out, label)

    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-51-076a29728661>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/124 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0672985315322876 train acc 0.46875
epoch 1 train acc 0.834971438172043


<ipython-input-51-076a29728661>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(dev_dataloader)):


  0%|          | 0/8 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6279721467391304


  0%|          | 0/124 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1562313288450241 train acc 0.96875
epoch 5 train acc 0.9649697580645161


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 5 test acc 0.6318783967391304


In [52]:
class BERTTestDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [53]:
test_data_list = []

for q in test_data['comments']:
    data = []
    data.append(q)

    test_data_list.append(data)

In [54]:
test_dataset = BERTTestDataset(test_data_list, 0, tok, vocab, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [55]:
model.eval()
test_label = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    valid_length = valid_length
    out = model(token_ids, valid_length)
    out = nn.Softmax(dim=1)(out)
    pred = torch.argmax(out, dim=-1)
    test_label.append(pred)

<ipython-input-55-8748a896eaea>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/974 [00:00<?, ?it/s]

In [56]:
test_label = [int(k.cpu()) for k in test_label]
test_data['label'] = test_label

In [57]:
test_data

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,1
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


In [58]:
test_data.to_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/submission_final3.csv', index=None)